In [16]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import plotly.express as px
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint
import branca.colormap as cm

In [17]:
# admin dasar
# admin_gdf = gpd.read_file('../data/SHP/Administrasi.shp')
# admin_gdf['centroid_latitude'] = admin_gdf['geometry'].centroid.y
# admin_gdf['centroid_longitude'] = admin_gdf['geometry'].centroid.x
# bintuni_dasar_gdf = admin_gdf.dissolve(by='Kabupaten').reset_index()
# bintuni_dasar_gdf

# sagu
topografi_gdf = gpd.read_file('../data/SHP/Topografi.shp')
# projected_crs = 'EPSG:3857' #Untuk Web Mercator (digunakan oleh OpenStreetMap, Google Maps, dan lainnya):
# sagu_gdf = sagu_gdf.to_crs(projected_crs)
topografi_gdf['centroid_latitude'] = topografi_gdf['geometry'].centroid.y
topografi_gdf['centroid_longitude'] = topografi_gdf['geometry'].centroid.x
# topografi_gdf = topografi_gdf.dissolve(by='Penggunaan').reset_index()
topografi_gdf['batas_awal']=topografi_gdf['Topografi'].apply(lambda x: x.split(' - ')[0]).astype(int)
topografi_gdf.sort_values(by='batas_awal', inplace=True)
topografi_gdf = topografi_gdf.drop(1).reset_index(drop=True) # drop 1 karena salah dari resky

tp_rendah = topografi_gdf[topografi_gdf['Dataran']=="Dataran Rendah"]
tp_tinggi = topografi_gdf[topografi_gdf['Dataran']=="Dataran Tinggi"]
topografi_gdf

/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_4612/1237062547.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  topografi_gdf['centroid_latitude'] = topografi_gdf['geometry'].centroid.y
/var/folders/b9/v73gyxys1djbgjy3c8c3cfwm0000gn/T/ipykernel_4612/1237062547.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  topografi_gdf['centroid_longitude'] = topografi_gdf['geometry'].centroid.x


,Topografi,Dataran,geometry,centroid_latitude,centroid_longitude,batas_awal
0,0 - 10,Dataran Rendah,"MULTIPOLYGON Z (((133.63269 -2.90712 0.00000, ...",-2.308229,133.386423,0
1,10 - 20,Dataran Rendah,"MULTIPOLYGON Z (((133.61257 -2.94449 0.00000, ...",-2.289481,133.329909,10
2,20 - 30,Dataran Rendah,"MULTIPOLYGON Z (((133.28496 -2.98916 0.00000, ...",-2.262199,133.269998,20
3,30 - 40,Dataran Rendah,"MULTIPOLYGON Z (((133.41953 -3.00194 0.00000, ...",-2.344345,133.296804,30
4,40 - 50,Dataran Rendah,"MULTIPOLYGON Z (((133.42805 -3.01453 0.00000, ...",-2.397522,133.354010,40
5,50 - 100,Dataran Rendah,"MULTIPOLYGON Z (((133.50413 -3.07320 0.00000, ...",-2.330461,133.378253,50
6,100 - 500,Dataran Rendah,"MULTIPOLYGON Z (((133.50718 -3.07091 0.00000, ...",-2.002399,133.606288,100
7,100 - 500,Dataran Tinggi,"MULTIPOLYGON Z (((134.25737 -3.07635 0.00000, ...",-2.041910,133.693224,100
8,500 - 1000,Dataran Tinggi,"MULTIPOLYGON Z (((134.23493 -3.01842 0.00000, ...",-1.781082,133.478802,500
9,1000 - 1500,Dataran Tinggi,"MULTIPOLYGON Z (((134.27279 -3.11490 0.00000, ...",-1.616237,133.424811,1000


In [18]:
colormap_rendah = cm.LinearColormap(
    ('#1b8515','#d4c92e'), 
    index=None,
    vmin=tp_rendah.batas_awal.min(), 
    vmax=tp_rendah.batas_awal.max(),
    caption="Ketinggian Tempat (mdpl)", 
    # max_labels=10, 
    tick_labels=tp_rendah.batas_awal.tolist()
)

colormap_tinggi = cm.LinearColormap(
    ('#d4c92e','#9d1e1e'), 
    index=None,
    vmin=tp_tinggi.batas_awal.min(), 
    vmax=tp_tinggi.batas_awal.max(),
    caption="Ketinggian Tempat (mdpl)", 
    # max_labels=10, 
    tick_labels=tp_tinggi.batas_awal.tolist()
)

# colormap_legend = cm.LinearColormap(
#     ('#1b8515','#d4c92e','#9d1e1e'), 
#     index=None,
#     vmin=tp_rendah.batas_awal.min(), 
#     vmax=tp_tinggi.batas_awal.max(),
#     caption="Ketinggian Tempat (mdpl)", 
#     # max_labels=10, 
#     tick_labels=(tp_rendah.batas_awal.tolist()+tp_tinggi.batas_awal.tolist())
# )
# colormap_legend.to_step(11)

In [19]:
m = folium.Map([-2.033975531897878, 133.346063060061], zoom_start=9)

folium.GeoJson(
    tp_rendah,
    name="Topografi Rendah",
    style_function=lambda feature: {
        "fillColor": colormap_rendah(feature['properties']['batas_awal']),
        "color": colormap_rendah(feature['properties']['batas_awal']),
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 1,
    },
    # smooth_factor=2,
).add_to(m)


folium.GeoJson(
    tp_tinggi,
    name="Topografi Tinggi",
    style_function=lambda feature: {
        "fillColor": colormap_tinggi(feature['properties']['batas_awal']),
        "color": colormap_tinggi(feature['properties']['batas_awal']),
        "weight": 0.5,
        "dashArray": "5, 5",
        "fillOpacity": 1,
    },
    # smooth_factor=2,
).add_to(m)

# colormap_legend.add_to(m)
colormap_rendah.add_to(m)
colormap_tinggi.add_to(m)
plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)
folium.LayerControl().add_to(m)

In [20]:
m.save('../map-html/topografi.html')